In [ ]:
#!pip install catboost

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", 200)

from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split, RandomizedSearchCV
#from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RepeatedKFold, cross_val_score, cross_validate
#from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.base import clone

In [2]:
df = pd.read_csv('housing_clean.csv', index_col=0)
df.shape

(2374, 98)

In [ ]:
df.info(verbose = True)

### Remove original columns that have been ordinalized

In [3]:
df = df.drop(columns=[col for col in df.columns if col.endswith('_todrop')])

### Handle nominal features
- For CatBoost, we don't need to dummify nominal features
- We do need to covert all nominal feature columns into string data type
- Better to do this before train / test split

In [4]:
categ_nom = ['MSSubClass','BldgType','HouseStyle','SaleType','SaleCondition','MiscFeature',
             'Fence','GarageType','RoofStyle','RoofMatl','exterior_covering','MasVnrType',
             'MSZoning','Street','LotShape','LandContour','LotConfig','LandSlope','Neighborhood',
             'Alley','condition','Foundation','Utilities','Heating',
             'CentralAir','MoSold','Electrical','QrtSold','YrSold']

In [5]:
# check data type for each nominal feature
print(df[categ_nom].dtypes)

MSSubClass            int64
BldgType             object
HouseStyle           object
SaleType             object
SaleCondition        object
MiscFeature          object
Fence                object
GarageType           object
RoofStyle            object
RoofMatl             object
exterior_covering    object
MasVnrType           object
MSZoning             object
Street               object
LotShape             object
LandContour          object
LotConfig            object
LandSlope            object
Neighborhood         object
Alley                object
condition            object
Foundation           object
Utilities            object
Heating              object
CentralAir           object
MoSold                int64
Electrical           object
QrtSold              object
YrSold                int64
dtype: object


In [6]:
# turn all nominal features into string type so it can be passed in cat_features argument later
df[categ_nom] = df[categ_nom].astype(str)

In [7]:
print(df[categ_nom].dtypes)

MSSubClass           object
BldgType             object
HouseStyle           object
SaleType             object
SaleCondition        object
MiscFeature          object
Fence                object
GarageType           object
RoofStyle            object
RoofMatl             object
exterior_covering    object
MasVnrType           object
MSZoning             object
Street               object
LotShape             object
LandContour          object
LotConfig            object
LandSlope            object
Neighborhood         object
Alley                object
condition            object
Foundation           object
Utilities            object
Heating              object
CentralAir           object
MoSold               object
Electrical           object
QrtSold              object
YrSold               object
dtype: object


### Perform Train / Test Split

In [43]:
X = df.drop(['SalePrice', 'PID'], axis = 1)
y = df['SalePrice']

In [8]:
# Split the data into train and test based on the year the house was sold
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

X_train = df.loc[df['YrSold'].astype(int).between(2006, 2009)].drop(['SalePrice', 'PID'], axis = 1)
X_test = df.loc[df['YrSold'].astype(int) == 2010].drop(['SalePrice', 'PID'], axis = 1)

y_train = df.loc[df['YrSold'].astype(int).between(2006, 2009)]['SalePrice']
y_test = df.loc[df['YrSold'].astype(int) == 2010]['SalePrice']

In [9]:
# below only used in early stopping model
# Further splitting the training set into train and validation set for hyperparam tuning
X_tra, X_val, y_tra, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state = 42)

### Setting up a baseline model before any tuning

* **Set a fair baseline:** Train a plain (untuned) model to get a reference R² you can compare against after tuning.
* **Reduce split luck:** Repeat cross validation multiple times for the training dataset and **average** the R² so your baseline isn’t driven by a single “easy” or “hard” split.
* **Isolate variability:** Fix the model’s `random_state` so differences come from the **data splits**, not extra model randomness.
* **Make comparisons honest:** Report mean ± std of train/test R² from these repeats; tuning “wins” only if it beats this baseline by more than the noise.

In [16]:
model_baseline = CatBoostRegressor(
    cat_features=categ_nom,
    verbose=0,  # Suppress training output
    random_state=42)

In [17]:
model_baseline.fit(X_train, y_train)

In [18]:
y_train_pred = model_baseline.predict(X_train)
y_test_pred = model_baseline.predict(X_test)

In [22]:
# R2
r2_baseline_train = model_baseline.score(X_train, y_train)
r2_baseline_test = model_baseline.score(X_test, y_test)

print(f"Train R square: {r2_baseline_train:.4f}")
print(f"Test R square: {r2_baseline_test:.4f}") 

Train R square: 0.9874
Test R square: 0.9319


In [23]:
# root mean squared error, lower is better
# sensitive to outliers
# tells you the average size of the error in prediction — in dollar terms
rmse_baseline_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_baseline_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Train Root Mean Squared Error: {rmse_baseline_train:.2f}")
print(f"Test Root Mean Squared Error: {rmse_baseline_test:.2f}")

Train Root Mean Squared Error: 8480.36
Test Root Mean Squared Error: 19397.18


In [10]:
# Implementing Repeated K-Fold Cross-Validation (before tuning any hyperparameters)

n_splits = 5
n_repeats = 3
rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

# Initialize CatBoost baseline model
model = CatBoostRegressor(
    cat_features=categ_nom,
    verbose=0,  # Suppress training output
    random_state=42)

# Lists to store the scores and RMSE values
scores = []
rmse_values = []

# Perform the cross-validation on the TRAINING data
for train_index, val_index in rkf.split(X_train):
    # Split the training data into internal training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training fold
    model.fit(X_train_fold, y_train_fold)

    # Evaluate the model on the validation fold
    score = model.score(X_val_fold, y_val_fold)
    scores.append(score)
    
    # Calculate RMSE manually
    y_pred_val = model.predict(X_val_fold)
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_val))
    rmse_values.append(rmse)

# Calculate the average score
average_score = np.mean(scores)
average_rmse = np.mean(rmse_values)
std_rmse = np.std(rmse_values)

print(f"Average baseline R-squared score: {average_score:.4f}")
print(f"Average baseline RMSE: ${average_rmse:.2f}")
print(f"Standard deviation of RMSE: ${std_rmse:.2f}")

Average baseline R-squared score: 0.9300
Average baseline RMSE: $19799.04
Standard deviation of RMSE: $3354.87


### Hyperparameter Tuning

### First, trying RandomizedSearchCV to find the best model
- The best R squared found is 86%, not ideal

In [54]:
# Setting parameter grid
param_grid = {
    "iterations": [500, 800, 1000],
    "depth": [4, 6, 8],
    "learning_rate": [0.01, 0.05, 0.1],
    "l2_leaf_reg": [1, 3, 5],
    "bagging_temperature": [0.5, 1, 2],
    "border_count": [32, 64, 128]
}

In [55]:
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=20,                # number of random combos to try
    #scoring='neg_root_mean_squared_error',
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

random_search.fit(X_train, y_train)

print("Best CV R-squared:", random_search.best_score_)
print("Best Parameters:", random_search.best_params_)

/Users/erinyu/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



Best CV R-squared: 0.8645086037381787
Best Parameters: {'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 800, 'depth': 6, 'border_count': 64, 'bagging_temperature': 0.5}


### Secondly, experimenting with Optuna

In [25]:
#!pip install optuna

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [optuna]2m4/5 [optuna]]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.12.0 requires pyarrow>=8.0.0, which is not installed.
edgartools 2.10.1 requires pyarrow>=14.0.0, which is not installed.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [26]:
import optuna
from optuna.samplers import TPESampler

In [34]:
# It's a good practice to set up logging to see Optuna's progress
optuna.logging.set_verbosity(optuna.logging.INFO)

#X_tra, X_val, y_tra, y_val
def objective(trial, X_tra, X_val, y_tra, y_val):
    """
    This function defines the search space for Optuna. It suggests hyperparameters,
    trains the model on the training set, evaluates it on the test set, and returns
    a single value to be optimized (R-squared score).

    Args:
        trial (optuna.trial.Trial): The trial object that suggests hyperparameters.
        X_tra (pd.DataFrame): Training feature matrix.
        X_val (pd.DataFrame): Testing feature matrix.
        y_tra (pd.Series): Training target vector.
        y_val (pd.Series): Testing target vector.

    Returns:
        float: The R-squared score on the test set.
    """
    # Define the search space for the hyperparameters using trial.suggest_...
    param = {
        # 'iterations': The number of boosting iterations (trees).
        'iterations': trial.suggest_int('iterations', 500, 1500, step=100),
        
        # 'depth': The depth of the trees.
        'depth': trial.suggest_int('depth', 4, 10),
        
        # 'learning_rate': The step size shrinkage.
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        
        # 'l2_leaf_reg': The L2 regularization coefficient.
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        
        # 'bagging_temperature': The bagging temperature for feature subsampling.
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.5, 5),
        
        # 'border_count': The number of splits for float features.
        'border_count': trial.suggest_int('border_count', 32, 255),
        
        # 'random_seed': Use a fixed seed for reproducibility across trials.
        'random_seed': 42,
        'verbose': 0
    }

    # Initialize the CatBoost Regressor with the parameters suggested by the trial
    model = CatBoostRegressor(**param, cat_features=categ_nom)

    # Train the model on the training data.
    # We use the test set as the evaluation set to allow for early stopping,
    # which can prevent overfitting and save time.
    model.fit(X_tra, y_tra, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=0)

    # Make predictions on the test data
    y_pred = model.predict(X_val)
    
    # Calculate the R-squared score on the test set
    r2 = r2_score(y_val, y_pred)

    # Optuna needs a single value to optimize, so we return the R-squared score
    return r2

In [35]:
# --- Create the Optuna study and run the optimization ---
# We want to maximize the R-squared score, so we set direction='maximize'
# We explicitly use TPESampler as requested.
study = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42)
)

print("Starting Optuna optimization for hyperparameter tuning...")
# Run the optimization for a specified number of trials.
# We use a lambda function to pass the data to the objective function.
study.optimize(lambda trial: objective(trial, X_train, X_test, y_train, y_test), n_trials=50)

# --- Print the results ---
print("-" * 30)
print("Optuna optimization completed.")
print(f"Number of finished trials: {len(study.trials)}")
print(f"Best R-squared score found: {study.best_value:.4f}")
print("\nBest parameters found:")
for param, value in study.best_params.items():
    print(f"  {param}: {value}")

[I 2025-09-11 15:36:52,239] A new study created in memory with name: no-name-175d886a-6373-4e1b-9101-8096049ccea3
/var/folders/4s/xd_3ly695k1gl73bn_9m85j40000gn/T/ipykernel_5317/2676129936.py:30: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



Starting Optuna optimization for hyperparameter tuning...


[I 2025-09-11 15:37:00,517] Trial 0 finished with value: 0.9212715013412772 and parameters: {'iterations': 900, 'depth': 10, 'learning_rate': 0.08960785365368121, 'l2_leaf_reg': 6.387926357773329, 'bagging_temperature': 1.2020838819909643, 'border_count': 66}. Best is trial 0 with value: 0.9212715013412772.
/var/folders/4s/xd_3ly695k1gl73bn_9m85j40000gn/T/ipykernel_5317/2676129936.py:30: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2025-09-11 15:37:08,910] Trial 1 finished with value: 0.9192064405393113 and parameters: {'iterations': 500, 'depth': 10, 'learning_rate': 0.06054365855469246, 'l2_leaf_reg': 7.372653200164409, 'bagging_temperature': 0.592630224331111, 'border_count': 249}. Best is trial 0 with value: 0.9212715013412772.
/var/folders/4s/xd_3ly695k1gl73bn_9m85j40000gn/T/ipykernel_5317/2676129936.py:30: FutureWarni

[I 2025-09-11 15:38:19,496] Trial 14 finished with value: 0.9299609900481027 and parameters: {'iterations': 1300, 'depth': 7, 'learning_rate': 0.03125863178903644, 'l2_leaf_reg': 1.087478880099385, 'bagging_temperature': 2.6163459486082, 'border_count': 197}. Best is trial 11 with value: 0.9312360120710792.
/var/folders/4s/xd_3ly695k1gl73bn_9m85j40000gn/T/ipykernel_5317/2676129936.py:30: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2025-09-11 15:38:25,311] Trial 15 finished with value: 0.9218582975347922 and parameters: {'iterations': 1300, 'depth': 8, 'learning_rate': 0.044785126644304896, 'l2_leaf_reg': 3.6396585625196165, 'bagging_temperature': 2.3719264768066717, 'border_count': 205}. Best is trial 11 with value: 0.9312360120710792.
/var/folders/4s/xd_3ly695k1gl73bn_9m85j40000gn/T/ipykernel_5317/2676129936.py:30: Future

[I 2025-09-11 15:39:42,262] Trial 28 finished with value: 0.9245352204693663 and parameters: {'iterations': 700, 'depth': 9, 'learning_rate': 0.03686274807234886, 'l2_leaf_reg': 1.6845620016573455, 'bagging_temperature': 2.115001713169667, 'border_count': 173}. Best is trial 11 with value: 0.9312360120710792.
/var/folders/4s/xd_3ly695k1gl73bn_9m85j40000gn/T/ipykernel_5317/2676129936.py:30: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2025-09-11 15:39:43,925] Trial 29 finished with value: 0.9264475751592268 and parameters: {'iterations': 1400, 'depth': 8, 'learning_rate': 0.11436167596055491, 'l2_leaf_reg': 6.1291492806676615, 'bagging_temperature': 1.2395464363677615, 'border_count': 93}. Best is trial 11 with value: 0.9312360120710792.
/var/folders/4s/xd_3ly695k1gl73bn_9m85j40000gn/T/ipykernel_5317/2676129936.py:30: Future

[I 2025-09-11 15:40:33,163] Trial 42 finished with value: 0.9258496341531612 and parameters: {'iterations': 800, 'depth': 6, 'learning_rate': 0.02371791352396891, 'l2_leaf_reg': 2.345891467926635, 'bagging_temperature': 4.934767807656291, 'border_count': 153}. Best is trial 38 with value: 0.9326364570911319.
/var/folders/4s/xd_3ly695k1gl73bn_9m85j40000gn/T/ipykernel_5317/2676129936.py:30: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2025-09-11 15:40:37,265] Trial 43 finished with value: 0.9379196460311489 and parameters: {'iterations': 1100, 'depth': 5, 'learning_rate': 0.018458591808163536, 'l2_leaf_reg': 2.0510760772316825, 'bagging_temperature': 3.2878093249289897, 'border_count': 81}. Best is trial 43 with value: 0.9379196460311489.
/var/folders/4s/xd_3ly695k1gl73bn_9m85j40000gn/T/ipykernel_5317/2676129936.py:30: Future

------------------------------
Optuna optimization completed.
Number of finished trials: 50
Best R-squared score found: 0.9379

Best parameters found:
  iterations: 1100
  depth: 5
  learning_rate: 0.018458591808163536
  l2_leaf_reg: 2.0510760772316825
  bagging_temperature: 3.2878093249289897
  border_count: 81


In [36]:
study.best_params

{'iterations': 1100,
 'depth': 5,
 'learning_rate': 0.018458591808163536,
 'l2_leaf_reg': 2.0510760772316825,
 'bagging_temperature': 3.2878093249289897,
 'border_count': 81}

### Hyperparameter Importances Plot
- This plot shows you which hyperparameters were most influential in determining the model's performance during the optimization process. The importance is calculated by analyzing how much each parameter's value impacted the final R-squared score.

- Essentially, if a hyperparameter like learning_rate or depth has a high importance score, it means that changes to its value led to significant changes in the model's performance. Conversely, a parameter with a low importance score means that varying its value didn't have much effect on the final outcome.

- This plot is incredibly useful for refining your next optimization. For instance, if you see that a certain hyperparameter has very low importance, you might be able to fix its value or remove it from the search space in a future run to make the optimization faster and more efficient.

In [37]:
optuna.visualization.plot_param_importances(study)

### Optimization History Plot
- This plot helps you understand if your optimization is making good progress or if it has reached its full potential.

- Trial-by-Trial Performance: Each dot on the plot represents a single trial (one set of hyperparameters) and its corresponding objective value (the R-squared score in your case). This shows you how the model performed with different parameter combinations.

- Progress Over Time: The x-axis represents the trial number. As you move from left to right, you can see how the performance of the trials changes over the course of the optimization.

- Convergence: The most important part of this plot is the "best value so far" line. This line shows the highest R-squared score found up to each trial. If this line starts to flatten out and doesn't improve much over the last few trials, it's a strong indicator that the optimization has converged. This can tell you that running more trials might not be necessary to find a better solution.

In [38]:
optuna.visualization.plot_optimization_history(study)

### Model Evaluation

In [41]:
# Retrain the model using the best hyperparameters found by Optuna
final_model = CatBoostRegressor(
    **study.best_params, 
    cat_features=categ_nom,
    verbose=0,
    random_seed=42
)

final_model.fit(X_train, y_train)

In [42]:
# Make predictions and evaluate on the test set
final_y_pred = final_model.predict(X_test)
final_r2 = r2_score(y_test, final_y_pred)
print(f"Final R-squared score on the test set: {final_r2:.4f}")

Final R-squared score on the test set: 0.9379


In [47]:
# --- Cross-Validation with Final Model ---
print("\n" + "="*50)
print("Performing K-fold Cross-Validation with the Final Model...")
print("="*50)

# Re-instantiate the final model with best parameters
final_model_cv = CatBoostRegressor(
    **study.best_params, 
    cat_features=categ_nom,
    verbose=0,
    random_seed=42
)

# Implementing Repeated K-Fold Cross-Validation (before tuning any hyperparameters)

n_splits = 5
n_repeats = 3
rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

scoring = {'R-squared': 'r2',
           'Neg-RMSE': 'neg_root_mean_squared_error'
          }


# Perform cross-validation and get the R-squared scores
# We use the full dataset (X, y) for cross-validation
cv_scores = cross_validate(final_model_cv, X, y, cv=rkf, scoring=scoring, n_jobs=-1)

# Print the results for each metric
print("\nCross-Validation Results:")
for metric_name, score_array in cv_scores.items():
    if metric_name.startswith('test_'):
        print(f"  {metric_name.replace('test_', '')} scores: {np.round(score_array, 4)}")
        print(f"  Mean {metric_name.replace('test_', '')} score: {np.mean(score_array):.4f}")
        print(f"  Standard deviation of {metric_name.replace('test_', '')} score: {np.std(score_array):.4f}")
        print("-" * 30)

print("\nFinal evaluation and analysis complete.")




Performing K-fold Cross-Validation with the Final Model...

Cross-Validation Results:
  R-squared scores: [0.9522 0.9277 0.9291 0.9582 0.8558 0.944  0.9379 0.8923 0.9223 0.9401
 0.9323 0.9518 0.9344 0.9355 0.8766]
  Mean R-squared score: 0.9260
  Standard deviation of R-squared score: 0.0280
------------------------------
  Neg-RMSE scores: [-17862.3119 -21847.897  -18287.5747 -15456.0335 -25903.5121 -19124.1616
 -18212.2379 -24580.9825 -21312.6407 -17406.0098 -20234.3589 -14736.5762
 -20551.7763 -19915.8185 -25334.5402]
  Mean Neg-RMSE score: -20051.0955
  Standard deviation of Neg-RMSE score: 3228.0497
------------------------------

Final evaluation and analysis complete.


In [50]:
pd.DataFrame(cv_scores).describe()

,fit_time,score_time,test_R-squared,test_Neg-RMSE
count,15.000000,15.000000,15.000000,15.000000
mean,14.816590,0.010737,0.926007,-20051.095450
std,2.503666,0.005691,0.029009,3341.348886
min,11.601601,0.003483,0.855846,-25903.512082
25%,13.240551,0.007393,0.925012,-21580.268818
50%,13.607336,0.008673,0.934385,-19915.818524
75%,17.116777,0.013254,0.942029,-18037.274870
max,19.244082,0.024350,0.958211,-14736.576209


In [53]:
from sklearn.dummy import DummyRegressor
# --- Baseline Model Evaluation ---
print("\n" + "="*50)
print("Performing Baseline Model Evaluation...")
print("="*50)

# Instantiate a simple baseline model (DummyRegressor)
# The default strategy for DummyRegressor is 'mean', which is a good baseline for regression.
baseline_model = DummyRegressor(strategy='mean')

# Perform cross-validation on the baseline model
# We use the same K-fold split and scoring metrics for a fair comparison
baseline_cv_results = cross_validate(baseline_model, X, y, cv=rkf, scoring=scoring, n_jobs=-1)

# Print the results for each metric
print("\nBaseline Model Cross-Validation Results:")
for metric_name, score_array in baseline_cv_results.items():
    if metric_name.startswith('test_'):
        print(f"  {metric_name.replace('test_', '')} scores: {np.round(score_array, 4)}")
        print(f"  Mean {metric_name.replace('test_', '')} score: {np.mean(score_array):.4f}")
        print(f"  Standard deviation of {metric_name.replace('test_', '')} score: {np.std(score_array):.4f}")
        print("-" * 30)


Performing Baseline Model Evaluation...

Baseline Model Cross-Validation Results:
  R-squared scores: [-0.0011 -0.004  -0.0003 -0.0005 -0.0115 -0.0067 -0.0005 -0.002  -0.0011
 -0.0011 -0.0092 -0.0154 -0.0002 -0.0008 -0.    ]
  Mean R-squared score: -0.0036
  Standard deviation of R-squared score: 0.0047
------------------------------
  Neg-RMSE scores: [-81709.2431 -81424.9371 -68705.7217 -75625.728  -68616.9367 -81084.8483
 -73074.8984 -74970.4466 -76501.7859 -71131.0589 -78120.648  -67672.4267
 -80241.9298 -78419.4459 -72107.2671]
  Mean Neg-RMSE score: -75293.8215
  Standard deviation of Neg-RMSE score: 4723.5454
------------------------------


### Evaluate on Test Set

In [ ]:
best_params = random_search.best_params_

final_model = CatBoostRegressor(
    **best_params,
    random_state=42,
    verbose=0,
    cat_features=categ_nom
)

In [ ]:
final_model.fit(
    X_tr, y_tr,
    eval_set=(X_val, y_val),
    early_stopping_rounds=50
)

In [ ]:
# root mean squared error, lower is better
# sensitive to outliers
# tells you the average size of the error in prediction — in dollar terms

y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)

final_rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
final_rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Final RMSE on the train set: ${final_rmse_train:.4f}")
print(f"Final RMSE on the test set: ${final_rmse_test:.4f}")

In [ ]:
# R2
r2_5kf_train = final_model.score(X_train, y_train)
r2_5kf_test = final_model.score(X_test, y_test)

In [ ]:
print(f"Train R square: {r2_5kf_train:.4f}")
print(f"Test R square: {r2_5kf_test:.4f}") 

In [ ]:
cb_feature_importance_final = pd.DataFrame({"feature": X.columns, 
                                            "importance_cb_5fk":final_model.feature_importances_})

In [ ]:
cb_feature_importance_final.sort_values("importance_cb_5fk", ascending = False, inplace=True);
cb_feature_importance_final.head(10)

In [ ]:
temp = cb_feature_importance_final.iloc[:10,]
plt.barh(temp["feature"], temp["importance_cb_5fk"]);

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test_pred, y_test, c='crimson')

plt.title('Tuned CatBoost')
p1 = max(max(y_test_pred), max(y_test))
p2 = min(min(y_test_pred), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Predictions', fontsize=15)
plt.ylabel('True Values', fontsize=15)
plt.axis('equal')
plt.show()

## Ditch below and add model evaluation session:

That's an excellent question. You're thinking about the stability and reliability of your final model, which is a critical part of the machine learning workflow.

You've already established a strong process for building and evaluating a single, well-tuned model. To go a step further and test its stability, you can use a technique called **bootstrap aggregation**, which is at the heart of models like Random Forest. You can also explore residual plots and other error metrics to gain a deeper understanding of your model's weaknesses.

---

### Method 1: Bootstrap Aggregation (Bootstrapping)

Bootstrapping is a resampling technique that involves repeatedly drawing random samples **with replacement** from your dataset. You can use this to train multiple models and see how stable your performance metric (like RMSE) is across these different training sets. This tells you how sensitive your model's performance is to the specific data it's trained on.

Here's how to do it:

1.  **Loop and Resample**: Create a loop that runs, say, 100 times.
2.  **Draw a Sample**: In each iteration, take a bootstrap sample of your training data. The size of this sample should be equal to the size of your original training data.
3.  **Train and Evaluate**: For each sample, train a new CatBoost model using the best hyperparameters you found earlier.
4.  **Store the Score**: Evaluate this new model on your held-out test set and store the RMSE.
5.  **Analyze the Distribution**: After the loop finishes, you'll have a distribution of 100 RMSE scores.

By analyzing the distribution of these scores, you can get:

* **Average Performance**: The mean of the distribution gives you an even more robust estimate of your model's expected performance.
* **Performance Stability**: The standard deviation of the scores tells you how much the model's performance varies with different training data. A small standard deviation indicates a very stable model.

### Method 2: Residual Analysis

Another powerful way to test the stability and understand the behavior of your final model is to perform a **residual analysis**. Residuals are the differences between the actual house prices and your model's predicted prices ($residuals = y_{actual} - y_{predicted}$).

By plotting the residuals, you can identify patterns where your model might be systematically making errors.

1.  **Plot Residuals vs. Predicted Values**: A common plot is a scatter plot with predicted values on the x-axis and residuals on the y-axis.
    * **Ideal Plot**: The points should be randomly scattered around the zero line, forming a uniform band. This indicates that your model's errors are random and not dependent on the predicted price.
    * **Warning Signs**: If you see a pattern (e.g., a "fanning out" shape where residuals get larger for higher-priced houses), it means your model is less accurate for certain price ranges. This could indicate a need for more data, different features, or a more complex model. 

2.  **Plot Residuals vs. Features**: Plotting residuals against your features can reveal if your model is consistently under- or over-predicting for specific feature values. For example, plotting residuals against `GrLivArea` might show that your model consistently under-predicts the price for very large homes.

---

### Other Metrics and Comparisons

Finally, beyond RMSE, you can use other metrics to get a more complete picture of your model's performance:

* **Mean Absolute Error (MAE)**: This metric is less sensitive to outliers than RMSE. It gives you the average absolute difference between predicted and actual prices. A combination of low RMSE and MAE suggests a robust model with few large errors.
* **R-squared ($R^2$)**: While not ideal for tuning, the final $R^2$ score on the test set is an excellent way to see how much of the variance in house prices is explained by your model.
* **Baseline Comparison**: Your initial instinct was spot-on. Compare your final model's RMSE directly to your baseline model's average cross-validated RMSE. This provides a clear, quantitative measure of the value added by your hyperparameter tuning.

### Residual Plot
- After training your final CatBoost model and using it to make predictions on the test set, you can generate a residual plot to visually inspect its performance. This type of plot is a valuable tool for understanding if your model's errors are random or if there are systematic patterns.

- The plot below shows the relationship between your model's predicted values and the residuals (the difference between the actual and predicted values).

- An ideal residual plot will show a random, uniform scatter of points around the horizontal line at y=0. This indicates that your model's errors are random and are not related to the predicted value. It means your model is not systematically over- or under-predicting for certain ranges of house prices.

- In contrast, if you see a discernible pattern in the plot (e.g., a fanning-out shape, a curve, or a clear concentration of points), it suggests that your model may have a bias. For example, a fanning-out pattern could mean that the model's errors increase as the predicted price gets higher, indicating a need for a more complex model or additional features to handle higher-priced homes.

In [ ]:
# Plot Residuals vs. Predicted Values

residuals = y_test - y_test_pred

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test_pred, residuals, alpha=0.5)
plt.title('Residuals vs. Predicted Values')
plt.xlabel('Predicted House Price')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='--')
plt.grid(True)
plt.show()
#plt.savefig('residuals_plot.png')

#print("Residuals plot has been generated and saved as 'residuals_plot.png'.")

### Distribution of actual house prices, y_test and y_test_pred
- Overlap: The more the two distributions overlap, the better your model's predictions align with the actual values. An ideal model's y_pred distribution will closely mimic the y_test distribution.

- Shape: The shapes of the two distributions should be similar. If the y_pred distribution is much narrower than the y_test distribution, it means your model is under-predicting the range of prices and is not confident in making predictions for the highest or lowest-priced homes.

- Skewness: If the actual data is skewed (e.g., towards higher prices), your predicted data should show a similar skew.

In [ ]:
# Calculate the mean and standard deviation
y_test_mean = np.mean(y_test)
y_test_std = np.std(y_test)
y_test_pred_mean = np.mean(y_test_pred)
y_test_pred_std = np.std(y_test_pred)

print("Actual Values (y_test) statistics:")
print(f"Mean: {y_test_mean:.4f}")
print(f"Standard Deviation: {y_test_std:.4f}")
print("\nPredicted Values (y_test_pred) statistics:")
print(f"Mean: {y_test_pred_mean:.4f}")
print(f"Standard Deviation: {y_test_pred_std:.4f}")

In [ ]:
# Plot the distributions of y_test and y_test_pred
plt.figure(figsize=(10, 6))
sns.histplot(y_test, kde=True, color='blue', label='Actual Values (y_test)')
sns.histplot(y_test_pred, kde=True, color='red', label='Predicted Values (y_test_pred)')
sns.histplot(y, kde=True, color='orange', label='Sales Prices (y)')

plt.title('Distribution of Actual vs. Predicted Values')
plt.xlabel('House Price')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.show()
#plt.savefig('distribution_plot.png')

#print("Distribution plot has been generated and saved as 'distribution_plot.png'.")

In [ ]:
# plot residual vs features

# Choose the feature you want to plot against
feature_to_plot = 'GrLivArea'  # Change this to your desired feature (e.g., 'GrLivArea')
feature_values = X_test[feature_to_plot]

# Create the scatter plot of residuals vs. a chosen feature
plt.figure(figsize=(10, 6))
plt.scatter(feature_values, residuals, alpha=0.5)
plt.title(f'Residuals vs. {feature_to_plot}')
plt.xlabel(f'{feature_to_plot} Value')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='--')
plt.grid(True)
plt.show()
#plt.savefig(f'residuals_vs_{feature_to_plot}.png')

#print(f"Residuals vs. {feature_to_plot} plot has been generated and saved.")

In [ ]:
from sklearn.utils import resample

# --- 3. Perform Bootstrapping to Assess Stability ---
n_bootstraps = 100 # Number of bootstrap samples to create
rmse_scores = []   # List to store the RMSE score for each bootstrap run

#print(f"Starting bootstrapping with {n_bootstraps} iterations...")

for i in range(n_bootstraps):
    # a. Create a bootstrap sample of the training data
    # resample() draws a random sample with replacement
    X_sample, y_sample = resample(X_train, y_train, replace=True, random_state=i)

    # b. Train a new CatBoost model on the bootstrap sample
    # The model uses the best_params found during hyperparameter tuning
    model = CatBoostRegressor(**best_params, verbose=0, random_state=42, cat_features=categ_nom)
    model.fit(X_sample, y_sample)

    # c. Evaluate the model on the held-out test set
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # d. Store the RMSE score
    rmse_scores.append(rmse)

    # Print progress every 10 iterations
    if (i + 1) % 10 == 0:
        print(f"Completed iteration {i + 1}/{n_bootstraps}")

# --- 4. Analyze the Distribution of RMSE Scores ---
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)
min_rmse = np.min(rmse_scores)
max_rmse = np.max(rmse_scores)

print("\n--- Bootstrapping Results ---")
print(f"Average RMSE from {n_bootstraps} runs: {mean_rmse:.4f}")
print(f"Standard Deviation of RMSE: {std_rmse:.4f}")
print(f"Minimum RMSE: {min_rmse:.4f}")
print(f"Maximum RMSE: {max_rmse:.4f}")

In [ ]:
# --- 5. Visualize the Distribution ---
plt.figure(figsize=(10, 6))
plt.hist(rmse_scores, bins=20, edgecolor='black', alpha=0.7)
plt.axvline(mean_rmse, color='r', linestyle='dashed', linewidth=2, label=f'Mean RMSE: {mean_rmse:.4f}')
plt.title(f'Distribution of RMSE Scores Across {n_bootstraps} Bootstrap Samples')
plt.xlabel('Root Mean Squared Error (RMSE)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(axis='y', alpha=0.75)
plt.show()

### SHAP
- A SHAP value tells you how much a particular feature pushes the model’s predicted house price up or down, compared to the average prediction across the dataset.

In [ ]:
# Say the average predicted price is $200,000
# If for House A, the SHAP value for GrLivArea is +15,000, 
# that means the model believes the GrLivArea of House A contributes +$15,000 to the price, 
# compared to the average. If for House B, the SHAP value is -5,000, 
# that means the GrLivArea of House B is lowering the prediction by $5,000 from the average.

In [ ]:
import shap

# Fit explainer on train data
explainer = shap.Explainer(final_model)

# Evaluate SHAP values on test set
shap_values = explainer.shap_values(X_test)

#This balances accurate attribution with realistic performance evaluation.

In [ ]:
# SHAP feature importance
mean_abs_shap_values = np.abs(shap_values).mean(axis=0)

# Create a DataFrame for easier viewing and sorting
shap_importance = pd.DataFrame({
    'feature': X.columns,
    'mean_abs_shap_cb_5fk': mean_abs_shap_values
})

# Sort in descending order
shap_importance_sorted = shap_importance.sort_values(by='mean_abs_shap_cb_5fk', ascending=False)

# Display
print(shap_importance_sorted.head(30))

# how to interpret:
# # On average, the feature OverallQual changes the model’s prediction by 14,566 units of currency (likely dollars), 
# either increasing or decreasing the predicted house price.

In [ ]:
# to visualize the mean abs of SHAP value using summary bar plot
shap.summary_plot(shap_values, X_test, plot_type='bar')

In [ ]:
# summary violin plot
shap.summary_plot(shap_values, X_test)

In [ ]:
# How to Read the Violin Plot
# ✅ 1. Width of the Violin (Horizontal Spread)

# Wider areas indicate more data points with similar SHAP values.

# This shows the distribution of how much that feature affects predictions across your dataset.

# ✅ 2. Color Gradient (Red → Blue)

# Each dot is colored by the actual value of the feature:

# Red = High feature value

# Blue = Low feature value

# This helps you understand the direction of the effect.

# For example, if red dots are mostly on the right (positive SHAP values), high values of that feature increase the prediction (e.g., higher price).

# If blue dots are on the left (negative SHAP values), low values decrease the prediction.

# ✅ 3. X-Axis (SHAP Value)

# Represents the magnitude and direction of impact on the model’s prediction.

# A value of +5000 means the feature adds $5,000 (for a house price model).

# A value of -3000 means it subtracts $3,000 from the prediction.

### SHAP Dependence Plot
- In a SHAP dependence plot, the x-axis shows the value of the feature you specified, and the y-axis shows the SHAP value (i.e., the feature’s contribution to the model output). But if you’re seeing a second feature encoded on the plot, that’s expected and by design. The second feature (by color) is there to show interactions. The SHAP dependence plot automatically colors the points by the most correlated feature with the one on the x-axis. This helps uncover interactions between features.

In [ ]:
shap.dependence_plot('GrLivArea', shap_values, X_test)

# if don't want to show the second y-axis
#shap.dependence_plot('GrLivArea', shap_values, x_test, interaction_index=None)

In [ ]:
# Select top 20 features
top_features = shap_importance_sorted.head(20).index.tolist()

for feature in top_features:
    plt.figure(figsize=(6, 4))  # Compact size
    
    shap.dependence_plot(
        feature,
        shap_values,
        X_test,
        #interaction_index=None,  # Auto-picks a good feature for color encoding
        show=False
    )
    plt.title(f"SHAP Dependency Plot: {feature}")
    plt.tight_layout()
    plt.show()
    
    # Save plot to file
#     filename = f"shap_dependency_{feature}.png"
#     plt.savefig(filename, dpi=150)
#     plt.close()

# how to interpret
# using OverallQual as an example,
# 1stFlrSF is the most correlated feature with Overall
# you can see larger homes with high OverallQual have even higher SHAP values (strong positive synergy).
# Conversely, small homes (blue dots) with high OverallQual may still not be valued as highly.

### Export lists for final model comparison

### Extract the list of features and features that each one is most correlated with

In [ ]:
# Compute correlation matrix for SHAP dependency plot coloring
#cor_matrix = X_test.corr()

In [ ]:
# most_correlated = {}

# for feature in x_test.columns:
#     # Exclude self-correlation
#     corrs = cor_matrix[feature].drop(labels=[feature])
    
#     # Drop NaNs
#     corrs = corrs.dropna()
    
#     # Get feature with highest absolute correlation
#     if not corrs.empty:
#         most_correlated_feature = corrs.abs().idxmax()
#         most_correlated[feature] = most_correlated_feature


In [ ]:
# correlation_df = pd.DataFrame.from_dict(most_correlated, orient='index', columns=['Most_Correlated_Feature'])
# correlation_df.reset_index(inplace=True)
# correlation_df.columns = ['feature', 'most_correlated_feature_xgb_5fk']

In [ ]:
# correlation_df

In [ ]:
# limit the above list to top 20 features ranked by mean abs SHAP values
# xgb_SHAP_feature_final = pd.merge(
#     shap_importance_sorted, 
#     correlation_df, 
#     how='left',
#     on = 'feature'
# )

# xgb_SHAP_feature_final

In [ ]:
#xgb_feature_importance_final